<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/05_Transformers_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Sentimiento usando Transformers preentrenados

Este notebook demuestra cómo usar modelos Transformer preentrenados para análisis de
sentimiento en español usando la biblioteca 'transformers' de Hugging Face y el
modelo BETO (BERT entrenado en español).

#1) Importar librerias necesarias

In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline

#2) Descargar y cargar el modelo BETO y su tokenizador

In [13]:
# Crear pipeline de análisis de sentimiento usando BETO
classifier = pipeline(
    task="sentiment-analysis",
    model="finiteautomata/beto-sentiment-analysis",
    tokenizer="finiteautomata/beto-sentiment-analysis",
    return_all_scores=True  # Para obtener todas las probabilidades
)

Device set to use cpu


#3) Realizar Predicciones

In [14]:
# Datos de ejemplo
sentences = [
    'Me gusta mucho este curso',
    'Estoy aburrido de la rutina diaria',
]

# Realizar predicciones
for sentence in sentences:
    result = classifier(sentence)
    scores = result[0]  # Obtenemos las probabilidades para cada clase

    # Encontrar la clase con mayor probabilidad
    max_score = max(scores, key=lambda x: x['score'])

    print(f"Frase: '{sentence}'")
    print("Probabilidades:")
    for score in scores:
        print(f"- {score['label']}: {score['score']:.4f}")
    print(f"Predicción final: {max_score['label']} ({max_score['score']:.4f})")
    print()

Frase: 'Me gusta mucho este curso'
Probabilidades:
- NEG: 0.0012
- NEU: 0.0004
- POS: 0.9984
Predicción final: POS (0.9984)

Frase: 'Estoy aburrido de la rutina diaria'
Probabilidades:
- NEG: 0.9966
- NEU: 0.0030
- POS: 0.0004
Predicción final: NEG (0.9966)



#4) Probar con nuevas frases

In [15]:
# Probar con nuevas frases
test_sentences = [
    "Este producto es excelente",
    "El servicio fue terrible"
]

for sentence in test_sentences:
    result = classifier(sentence)
    scores = result[0]

    max_score = max(scores, key=lambda x: x['score'])

    print(f"Frase: '{sentence}'")
    print("Probabilidades:")
    for score in scores:
        print(f"- {score['label']}: {score['score']:.4f}")
    print(f"Predicción final: {max_score['label']} ({max_score['score']:.4f})")
    print()

Frase: 'Este producto es excelente'
Probabilidades:
- NEG: 0.0012
- NEU: 0.0004
- POS: 0.9984
Predicción final: POS (0.9984)

Frase: 'El servicio fue terrible'
Probabilidades:
- NEG: 0.9992
- NEU: 0.0004
- POS: 0.0005
Predicción final: NEG (0.9992)



##5) Realizar un ajuste fino (fine-tuning)

Vamos a mejorar el modelo BETO con nuestros propios datos
mediante un proceso de ajuste fino (fine-tuning).

In [17]:
!pip install datasets
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
import numpy as np
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Preparar nuestros datos de entrenamiento:

In [27]:
train_texts = [
    'Me gusta mucho este curso',
    'Este producto es excelente',
    'La calidad es increíble',
    'El servicio fue magnífico',
    'Me encanta la atención recibida',
    'La experiencia fue normal',
    'El producto cumple su función',
    'Estoy aburrido de la rutina diaria',
    'No estoy satisfecho con el servicio',
    'La calidad es pésima',
    'El servicio fue terrible',
    'No recomiendo este producto'
]

# Etiquetas: 0 (NEG), 1 (NEU), 2 (POS)
train_labels = [2, 2, 2, 2, 2, 1, 1, 0, 0, 0, 0, 0]

# 2. Crear dataset
dataset_dict = {
    'text': train_texts,
    'label': train_labels
}
train_dataset = Dataset.from_dict(dataset_dict)

Cargar modelo y tokenizador:

In [28]:
model_name = "finiteautomata/beto-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Preparar los datos:

In [29]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Ajustar el modelo mediante entrenamiento:

In [30]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

# 6. Crear y ejecutar el entrenamiento
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Entrenar el modelo
trainer.train()


Step,Training Loss


TrainOutput(global_step=9, training_loss=0.18142472373114693, metrics={'train_runtime': 61.1231, 'train_samples_per_second': 0.589, 'train_steps_per_second': 0.147, 'total_flos': 148001297472.0, 'train_loss': 0.18142472373114693, 'epoch': 3.0})

Probar el modelo ajustado:

In [31]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    prediction = torch.argmax(probabilities, dim=1).item()
    confidence = torch.max(probabilities).item()

    # Mapear predicción a etiqueta
    sentiment_map = {0: "NEG", 1: "NEU", 2: "POS"}
    sentiment = sentiment_map[prediction]

    return sentiment, confidence

# Probar con nuevas frases
test_sentences = [
    "La película fue increíble",
    "El servicio al cliente fue deficiente",
    "No me gustó nada la experiencia",
    "La comida estaba bien, normal",
    "El producto cumple con lo esperado"
]

print("\n=== Probando el modelo ajustado ===")
for sentence in test_sentences:
    sentiment, confidence = predict_sentiment(sentence)
    print(f"\nFrase: '{sentence}'")
    print(f"Sentimiento: {sentiment}")
    print(f"Confianza: {confidence:.4f}")


=== Probando el modelo ajustado ===

Frase: 'La película fue increíble'
Sentimiento: POS
Confianza: 0.9984

Frase: 'El servicio al cliente fue deficiente'
Sentimiento: NEG
Confianza: 0.9990

Frase: 'No me gustó nada la experiencia'
Sentimiento: NEG
Confianza: 0.9993

Frase: 'La comida estaba bien, normal'
Sentimiento: NEU
Confianza: 0.9929

Frase: 'El producto cumple con lo esperado'
Sentimiento: POS
Confianza: 0.9928
